# Winning Prediction

In this notebook, we try to predict the outcome of a league of legends game through analyzing winsOver relationships paths in a knowledge graph. 

## Idea 

For every match we search for the pairs on the lanes (the two top-laners, the two supports, ...) 
and search the shortest path of (a:Player)-\[WINS_OVER...\]->(b:Player) between these players.   
The longer the path, the higher we expect this player to win against the other player.  

<br><div><img src="../assets/match_visualization.png" width="500"/></div>

Then, we check for every lane who we expect to be the winner and determine the team with more wins to win overall. 

As an example: In the following image,Team B wins the jungle, the bot-lane and the support. 
So its 3/5. Hence,  we expect team B to win. 

<br><div><img src="../assets/winner_looser_majority_voting.png" width="500"/></div>


In [30]:
from sqlalchemy import Engine
from tqdm import tqdm

from db.graph import Neo4JConnector
from db.postgres import PostgreSQLConnector

import pandas as pd

In [31]:
neo4j: Neo4JConnector = Neo4JConnector.create_from_config("../config.ini")
engine: Engine = PostgreSQLConnector.create_from_config("../config.ini").create_engine()

### Loading the Data

First, we load the matches and fetch the players position from the knowledge graph.

Then we transform the match in an easy format. 

In [32]:
# Load matches that are not covered in the knowledge graph 
df = pd.read_sql("SELECT * FROM challenger_matches WHERE NOT is_train", con=engine)
df

cmid     id           game_creation game_duration     game_id  \
0      37793  37793 2020-02-26 09:43:52.081      00:26:38  4178138983   
1      37795  37795 2020-02-25 18:29:59.193      00:18:03  4177015257   
2      37800  37800 2020-02-25 14:03:21.394      00:38:12  4176387241   
3      37804  37804 2020-02-25 10:39:18.142      00:23:01  4175970993   
4      37816  37816 2020-03-11 14:37:07.470      00:35:38  4211450407   
...      ...    ...                     ...           ...         ...   
29174  37776  37776 2020-03-13 04:24:31.312      00:38:34  4214304469   
29175  37779  37779 2020-03-10 19:41:46.709      00:21:21  4209521684   
29176  37780  37780 2020-03-09 19:37:21.110      00:19:25  4207116799   
29177  37785  37785 2020-02-29 12:41:26.388      00:17:23  4185596991   
29178  37786  37786 2020-02-27 14:19:49.533      00:28:39  4181033984   

      game_mode     game_type   game_version  map_id  \
0       CLASSIC  MATCHED_GAME  10.4.308.9400      11   
1       CLASSIC  MATCHED_GAME  10.4.308.9400      11   
2       CLASSIC  MATCHED_GAME  10.4.308.9400      11   
3       CLASSIC  MATCHED_GAME  10.4.308.9400      11   
4       CLASSIC  MATCHED_GAME   10.5.312.392      11   
...         ...           ...            ...     ...   
29174   CLASSIC  MATCHED_GAME   10.5.312.392      11   
29175      ARAM  MATCHED_GAME   10.5.312.392      12   
29176   CLASSIC  MATCHED_GAME   10.5.311.166      11   
29177   CLASSIC  MATCHED_GAME  10.4.308.9400      11   
29178   CLASSIC  MATCHED_GAME  10.4.308.9400      11   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [33]:
# Load the players and receive their positions
query = """MATCH (p:Player {accountID: $account, platformID: $platform})-[hp:HAS_POSITION]->(pos:Position) RETURN pos.positionID"""

matches = {}
for _, row in tqdm(df.iterrows()):
    match = {"winning_team": {}, "loosing_team": {}}
    
    for participants, participant_identities in zip(row["participants"], row["participant_identities"]):
        win = participants["stats"]["win"]
        account, platform = participant_identities["player"]["accountId"], participant_identities["player"]["platformId"] 
        res = neo4j.exec(query, account=account, platform=platform)
        try:
            position = res.records[0]["pos.positionID"]
        except IndexError:
            continue
        
        match["winning_team" if win else "loosing_team"][position] = (account, platform)
    
    if len(match["winning_team"]) != 5 or len(match["loosing_team"]) != 5:
        continue
        
    matches[row['game_id']] = match

# visualize one match
matches[next(iter(matches))]

29179it [02:18, 210.43it/s]


{'winning_team': {'TOP': ('lGMaaj2xg-zRVzzfr464V5zNtX6wCZ0dyiLxO5DxAZZ-BGSv9x77gsc5',
   'KR'),
  'BOT': ('ORwB2bB0omL8TKjtQ9I-PO9EVA9yy35vbyv5LmMHC0Y7gyg', 'KR'),
  'SUP': ('k6ehq1OwIzkxwENgVSR0BS6KZs4zf3_j12s4cSVsf0rVkzdcrvSgWQez', 'KR'),
  'MID': ('Dli4ZNwRdA8FOkKO-i0E7qK7YHa4MucKqyHCeXFhCMf2RRc', 'KR'),
  'JGL': ('_4tnFamrTimgxNcSkAsiQzUY-IZ634GQgK4rMaqcQv64EyUow7xzgWP3', 'KR')},
 'loosing_team': {'SUP': ('CVT1n2uhHgp-k3rXp7y8oTV-IjR-PG7SKbRKiYO-8Tmp',
   'KR'),
  'MID': ('drHMdD3Vg_4Bc8GwQzPHck_uDsgYZz0rFoFK1IiBBWZYI_fZZRqGEzEY', 'KR'),
  'JGL': ('jQWGBs5KvBf3CH4QGlW819hovvEQQisaCCaGukgDDwQtNkxHyG9K35FD', 'KR'),
  'TOP': ('o_Q6xqmxPriZxRhQBNx7GXMUH9-dPVuXp8u2-o6M0ARd', 'KR'),
  'BOT': ('w6dam8aGMhUHMgDNGhwv6cncqqTd_7U8pEkg289syfMwvz1yb699xu3S', 'KR')}}

### Predicting 

In this part, we search for a path between the players on the positions and perform the majority voting on the win. 

In [2]:
import numpy as np

def get_size_save(arr):
    return 0 if len(arr) == 0 else len(arr[0]["scores"])

# Query to extract the path from the knowledge graph
query = '''MATCH path = shortestPath(
            (p1:Player {accountID: $p1accountid, platformID: $p1platformid})
            -[w:WINS_OVER *]->
            (p2:Player {accountID: $p2accountid, platformID: $p2platformid}))
        RETURN [r in relationships(path) | r.rate] AS scores'''


correct_classified = []
for match in tqdm(matches):
    # Estimated probability, that the winning team actually wins 
    prop_win = 0
    
    for position in ["TOP", "MID", "JGL", "SUP", "BOT"]:
        p_winner = matches[match]["winning_team"][position]
        p_looser = matches[match]["loosing_team"][position]
        
        # get the shortest path for the winning player
        result_winner = neo4j.exec(query, p1accountid=p_winner[0], p1platformid=p_winner[1], p2accountid=p_looser[0], p2platformid=p_looser[1])
        # get the shortest path for the loosing player
        result_looser = neo4j.exec(query, p1accountid=p_looser[0], p1platformid=p_looser[1], p2accountid=p_winner[0], p2platformid=p_winner[1])
        
        if len(result_winner.records) != 0 or len(result_looser.records) != 0: 
            path_size_winner, path_size_looser = get_size_save(result_winner.records), get_size_save(result_looser.records)
            
            # if path of the winner is greater than the loosers path, add one to the winning probability
            if get_size_save(path_size_winner.records) > get_size_save(path_size_looser.records):
                prop_win += 1
            # if path of the looser is greater than the winners path, subtract one from the winning probability
            elif get_size_save(path_size_winner.records) < get_size_save(path_size_looser.records):
                prop_win += -1
    
    # No winner can be determined
    if prop_win == 0:
        continue
    
    # If more lanes of the winning team won -> predict a win
    correct_classified.append(prop_win > 0) 
    
number_classification = np.size(correct_classified)
correct_classification = np.sum(correct_classified)

f"Correctly Classified: {correct_classification} / {number_classification} ({round(correct_classification * 100 / number_classification, 2)}%)"

NameError: name 'tqdm' is not defined